### Creación del servicio del productor

El objetivo de esta libreta es leer los tweets en tiempo real los cuáles son ingresados a Apache Kafka. Los pasos se describen a continuación:

- Se importan todas las bibliotecas necesarias para crear una conexión con Twitter, se lee el tweet y se mantiene disponible para la transmisión.
- Se lee el archivo JSON del tweet entrante (los tweets entrantes están en formato JSON).
- Se recupera solo el tweet real y se envia al socket del cliente.
- Se define el host y el puerto y se vinculan con el servicio.
- Se establece la conexión.
- Para conseguir la conexión en tiempo real: se usan las llaves de autenticación que proporciona Twitter.
- Se filtran los tweets con un tema específico.

In [0]:
!pip install kafka-python
!pip install python-twitter
!pip install --user tweepy

     |████████████████████████████████| 246 kB 6.6 MB/s 
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-28f745f0-58f9-4fd6-b29a-1e881e01fd4b/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 67 kB 3.3 MB/s 
     |████████████████████████████████| 491 kB 4.6 MB/s 
     |████████████████████████████████| 151 kB 92.9 MB/s 
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-28f745f0-58f9-4fd6-b29a-1e881e01fd4b/bin/python -m pip install --upgrade pip' command.
ERROR: Can not perform a '--user' install. User site-packages are not visible in this virtualenv.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-28f745f0-58f9-4fd6-b29a-1e881e01fd4b/bin/python -m pip install --upgrade pip' command.


In [0]:
!pip install tweepy

     |████████████████████████████████| 98 kB 1.8 MB/s 
  Attempting uninstall: requests-oauthlib
    Found existing installation: requests-oauthlib 2.0.0
    Uninstalling requests-oauthlib-2.0.0:
      Successfully uninstalled requests-oauthlib-2.0.0
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-28f745f0-58f9-4fd6-b29a-1e881e01fd4b/bin/python -m pip install --upgrade pip' command.


In [0]:
#Para conectarnos a twitter, se añaden las claves de acceso
access_token = "2896768435-CAgPFnvQn6HIAnUZr49tcT1BqLPuVNhH08pfP7f"
access_token_secret = "0K5xDayKIpmBvxH4lvbPxDVHSQZFKIhOHIBnNWnWhuW8g"
api_key = "2iymymMNxJzhXRbL0GU1LkZSE"
api_secret = "kLFtDdjamRbxhbBOz2v4duvg8FVvKTG0jQA8abCK0zpXrRyOum"

In [0]:
#Creación de la clase
import tweepy
from kafka import KafkaProducer
import json

In [0]:
%sh
curl -u "2iymymMNxJzhXRbL0GU1LkZSE:kLFtDdjamRbxhbBOz2v4duvg8FVvKTG0jQA8abCK0zpXrRyOum" \
  --data 'grant_type=client_credentials' \
  'https://api.x.com/oauth2/token'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   134  100   105  100    29    549    151 --:--:-- --:--:-- --:--:--   701
{"errors":[{"code":99,"message":"Unable to verify your credentials","label":"authenticity_token_error"}]}

In [0]:
!pip install TwitterAPI

  Created wheel for TwitterAPI: filename=TwitterAPI-2.8.2-py3-none-any.whl size=14896 sha256=96c6cd2dc9ba9dac3a2756a50c3a4f3b6739cd57eea96ded5f42cf10d07b188e
  Stored in directory: /root/.cache/pip/wheels/1d/03/fd/b8439407e01e9754cb844c5d024fb82415d61f24109f9f0db1
Successfully built TwitterAPI
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-28f745f0-58f9-4fd6-b29a-1e881e01fd4b/bin/python -m pip install --upgrade pip' command.


In [0]:
access_token = "2896768435-CAgPFnvQn6HIAnUZr49tcT1BqLPuVNhH08pfP7f"
access_token_secret = "0K5xDayKIpmBvxH4lvbPxDVHSQZFKIhOHIBnNWnWhuW8g"
api_key = ""
api_secret = "kLFtDdjamRbxhbBOz2v4duvg8FVvKTG0jQA8abCK0zpXrRyOum"

client = tweepy.Client(consumer_key=CONSUMER_API_KEY, 
                       consumer_secret=CONSUMER_API_SECRET, 
                       access_token=TWITTER_ACCESS_TOKEN, 
                       access_token_secret=TWITTER_TOKEN_SECRET) 

In [0]:
import tweepy

# API keys
access_token = '2896768435-CAgPFnvQn6HIAnUZr49tcT1BqLPuVNhH08pfP7f'
access_token_secret = '0K5xDayKIpmBvxH4lvbPxDVHSQZFKIhOHIBnNWnWhuW8g'
consumer_key = '2iymymMNxJzhXRbL0GU1LkZSE'
consumer_secret = 'kLFtDdjamRbxhbBOz2v4duvg8FVvKTG0jQA8abCK0zpXrRyOum'

# Auth set
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Obj creation
api = tweepy.API(auth)

# Topic set
query = "python"  # topic set
tweets = api.search_tweets(q=query, count=10)  # Download 10 tweets

# Showing tweets
for tweet in tweets:
    print(f"{tweet.user.screen_name}: {tweet.text}")

---------------------------------------------------------------------------
Unauthorized                              Traceback (most recent call last)
File <command-10923988045406>:18
     16 # Topic set
     17 query = "python"  # topic set
---> 18 tweets = api.search_tweets(q=query, count=10)  # Download 10 tweets
     20 # Showing tweets
     21 for tweet in tweets:

File /local_disk0/.ephemeral_nfs/envs/pythonEnv-28f745f0-58f9-4fd6-b29a-1e881e01fd4b/lib/python3.9/site-packages/tweepy/api.py:33, in pagination.<locals>.decorator.<locals>.wrapper(*args, **kwargs)
     31 @functools.wraps(method)
     32 def wrapper(*args, **kwargs):
---> 33     return method(*args, **kwargs)

File /local_disk0/.ephemeral_nfs/envs/pythonEnv-28f745f0-58f9-4fd6-b29a-1e881e01fd4b/lib/python3.9/site-packages/tweepy/api.py:46, in payload.<locals>.decorator.<locals>.wrapper(*args, **kwargs)
     44 kwargs['payload_list'] = payload_list
     45 kwargs['payload_type'] = payload_type
---> 46 return method(*arg

In [0]:
----------------------------

In [0]:
access_token = '2896768435-CAgPFnvQn6HIAnUZr49tcT1BqLPuVNhH08pfP7f'
access_token_secret = '0K5xDayKIpmBvxH4lvbPxDVHSQZFKIhOHIBnNWnWhuW8g'
consumer_key = '2iymymMNxJzhXRbL0GU1LkZSE'
consumer_secret = 'kLFtDdjamRbxhbBOz2v4duvg8FVvKTG0jQA8abCK0zpXrRyOum'

In [0]:
from TwitterAPI import TwitterAPI
api = TwitterAPI(consumer_key,
                 consumer_secret, access_token, access_token_secret)
r = api.request('search/tweets', {'q':'pizza'})
for item in r:
        print(item)

In [0]:
import tweepy
import pandas as pd

access_token = "2896768435-CAgPFnvQn6HIAnUZr49tcT1BqLPuVNhH08pfP7f"
access_token_secret = "0K5xDayKIpmBvxH4lvbPxDVHSQZFKIhOHIBnNWnWhuW8g"
consumer_key = "2iymymMNxJzhXRbL0GU1LkZSE"
consumer_secret = "kLFtDdjamRbxhbBOz2v4duvg8FVvKTG0jQA8abCK0zpXrRyOum"

#Pass in our twitter API authentication key
auth = tweepy.OAuth1UserHandler(consumer_key, consumer_secret,
                                access_token, access_token_secret)

#Instantiate the tweepy API
api = tweepy.API(auth, wait_on_rate_limit=True)

search_query = "'ref''world cup'-filter:retweets AND -filter:replies AND -filter:links"
no_of_tweets = 100

try:
    #The number of tweets we want to retrieved from the search
    tweets = api.search_tweets(q=search_query, lang="en", count=no_of_tweets, tweet_mode ='extended')
    
    #Pulling Some attributes from the tweet
    attributes_container = [[tweet.user.name, tweet.created_at, tweet.favorite_count, tweet.source, tweet.full_text] for tweet in tweets]

    #Creation of column list to rename the columns in the dataframe
    columns = ["User", "Date Created", "Number of Likes", "Source of Tweet", "Tweet"]
    
    #Creation of Dataframe
    tweets_df = pd.DataFrame(attributes_container, columns=columns)
except BaseException as e:
    print('Status Failed On,',str(e))

In [0]:
import tweepy

# Set up authentication
access_token = "2896768435-CAgPFnvQn6HIAnUZr49tcT1BqLPuVNhH08pfP7f"
access_token_secret = "0K5xDayKIpmBvxH4lvbPxDVHSQZFKIhOHIBnNWnWhuW8g"
consumer_key = "2iymymMNxJzhXRbL0GU1LkZSE"
consumer_secret = "kLFtDdjamRbxhbBOz2v4duvg8FVvKTG0jQA8abCK0zpXrRyOum"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Create API object
api = tweepy.API(auth)

# Define search query and number of tweets
query = 'python'
num_tweets = 100

# Extract tweets
tweets = tweepy.Cursor(api.search_tweets, q=query).items(num_tweets)

# Print tweets
for tweet in tweets:
    print(tweet.text)

In [0]:
import tweepy
# Define a class inheriting from StreamingClient to handle the incoming stream of tweets
class MyListener(tweepy.StreamingClient): 
    def on_tweet(self, tweet): # Use on_tweet to process incoming tweets
        print(tweet.text)

# Authenticate to the Twitter API using your credentials (replace placeholders with your actual keys)
#producer = KafkaProducer(bootstrap_servers='localhost:9092')
bearer_token = "AAAAAAAAAAAAAAAAAAAAACs6wQEAAAAA6d7eRr%2B9UtoD5UgFPHUPIDg4p%2Bw%3DeH6TaXyn1Bm8PPs0Swzr53bD3T1zxla2CXKajRfvQOUf4vTwl5"
twitter_stream = MyListener(bearer_token)
twitter_stream.filter(track=['#ucrania'])


In [0]:
class MyListener(tweepy.Stream):
    def on_data(self, data):
        try:
            with open('python.json', 'a') as f:
                #f.write(data)
                json_ = json.loads(data)
                print(json_['text'].encode('utf-8'))
                producer.send("test", json_["text"].encode('utf-8'))
                return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True
 
    def on_error(self, status):
        print(status)
        return True

producer = KafkaProducer(bootstrap_servers='localhost:9092')
twitter_stream = MyListener(api_key, api_secret,access_token, access_token_secret)
twitter_stream.filter(track=['#ucrania'])